In [2]:
import random
import numpy as np
import tensorflow as tf
import os
import json
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [3]:
# Enable Mixed Precision for Faster Training
tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [4]:
# Set seeds for reproducibility
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)


In [5]:
# Enable Apple Metal GPU acceleration
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


In [6]:
# Enable XLA Optimization
tf.config.optimizer.set_jit(True)

In [7]:
# Image Parameters
img_size = 224  # Reduced size for efficiency
batch_size = 16  # Reduced batch size to prevent memory issues

In [8]:
# Dataset Path
base_dir = '/Users/bhaveshreddy/Desktop/Capstone Project/plantvillage dataset'


In [9]:
# Check if dataset folders exist
print("Dataset Folders:", os.listdir(base_dir))

# Image Data Generators (Optimized augmentations)
data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,  # Reduced to avoid over-distortion
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2  # 20% validation split
)


Dataset Folders: ['grayscale', '.DS_Store', 'segmented', 'color']


In [10]:
# Train Generator
train_generator = data_gen.flow_from_directory(
    os.path.join(base_dir, "color"),
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'
)

Found 43456 images belonging to 38 classes.


In [11]:
# Validation Generator
validation_generator = data_gen.flow_from_directory(
    os.path.join(base_dir, "color"),
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'
)

Found 10849 images belonging to 38 classes.


In [12]:
# Custom ShuffleNetV2 Model
def ShuffleNetV2(input_shape=(224, 224, 3), num_classes=10):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(24, (3, 3), strides=(2, 2), padding='same', activation='relu')(inputs)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    
    for _ in range(3):  # Reduce complexity by having fewer stages
        x = layers.Conv2D(48, (1, 1), activation='relu')(x)
        x = layers.DepthwiseConv2D((3, 3), strides=(1, 1), padding='same', activation='relu')(x)
        x = layers.Conv2D(96, (1, 1), activation='relu')(x)
        x = layers.Add()([x, x])
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)
    
    model = models.Model(inputs, outputs)
    return model

In [13]:
# Initialize Model
num_classes = len(train_generator.class_indices)
model = ShuffleNetV2(input_shape=(img_size, img_size, 3), num_classes=num_classes)

# Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1)
checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_accuracy', mode='max')

# Train the Model
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=15,
    callbacks=[early_stopping, lr_scheduler, checkpoint]
)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 0s 779ms/step - accuracy: 0.1963 - loss: 3.0263

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2716/2716 ━━━━━━━━━━━━━━━━━━━━ 2586s 941ms/step - accuracy: 0.1963 - loss: 3.0262 - val_accuracy: 0.4066 - val_loss: 2.1607 - learning_rate: 0.0010
Epoch 2/15
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 2482s 914ms/step - accuracy: 0.4244 - loss: 2.0140 - val_accuracy: 0.6222 - val_loss: 1.2551 - learning_rate: 0.0010
Epoch 3/15
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 2686s 989ms/step - accuracy: 0.6236 - loss: 1.2482 - val_accuracy: 0.7228 - val_loss: 0.8733 - learning_rate: 0.0010
Epoch 4/15
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 2561s 942ms/step - accuracy: 0.7151 - loss: 0.9103 - val_accuracy: 0.7982 - val_loss: 0.6324 - learning_rate: 0.0010
Epoch 5/15
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 2288s 842ms/step - accuracy: 0.7708 - loss: 0.7355 - val_accuracy: 0.8188 - val_loss: 0.5550 - learning_rate: 0.0010
Epoch 6/15
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 2318s 853ms/step - accuracy: 0.7934 - loss: 0.6389 - val_accuracy: 0.8506 - val_loss: 0.4546 - learning_rate: 0.0010
Epoch 7/15
2716/2716 ━━━━━━━━━━━━━━━━━━━━ 2375s 874ms/ste

In [14]:
# Function to Load and Preprocess the Image
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = Image.open(image_path).convert('RGB')
    img = img.resize(target_size, Image.Resampling.LANCZOS)
    img_array = np.array(img).astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Expand batch dimension
    return img_array


In [16]:
# Function to Predict the Class of an Image
def predict_image_class(model, image_path, class_indices):
    preprocessed_img = load_and_preprocess_image(image_path)
    predictions = model.predict(preprocessed_img)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    class_indices = {v: k for k, v in class_indices.items()}  # Reverse mapping
    predicted_class_name = class_indices.get(predicted_class_index, "Unknown")
    return predicted_class_name


In [17]:
# Save class indices to JSON
with open('class_indices.json', 'w') as f:
    json.dump(train_generator.class_indices, f)

# Load class indices from JSON
with open('class_indices.json', 'r') as f:
    loaded_class_indices = json.load(f)

In [19]:
# Example Usage
image_path = '/Users/bhaveshreddy/Desktop/cardio-risk-prediction/checking.JPG'
predicted_class_name = predict_image_class(model, image_path, loaded_class_indices)
print("Predicted Class Name:", predicted_class_name)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
Predicted Class Name: Apple___Black_rot


In [22]:
import os

# Define the save path for .h5 and .keras
save_dir = '/Users/bhaveshreddy/Desktop/saved'
os.makedirs(save_dir, exist_ok=True)

# Save the model in both formats
model.save(os.path.join(save_dir, 'plant_disease_model.keras'))
model.save(os.path.join(save_dir, 'plant_disease_model.h5'))

print(f"Model saved at: {save_dir}/plant_disease_model.keras")
print(f"Model saved at: {save_dir}/plant_disease_model.h5")


Model saved at: /Users/bhaveshreddy/Desktop/saved/plant_disease_model.keras
Model saved at: /Users/bhaveshreddy/Desktop/saved/plant_disease_model.h5
